In [1]:
import pandas as pd
import numpy as np
import jellyfish
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
raw = pd.read_csv("test.csv", index_col=[0,5])

In [3]:
raw.columns

Index(['business_name', 'business_address', 'business_city', 'business_state',
       'business_latitude', 'business_longitude', 'business_phone_number',
       'inspection_id', 'inspection_date', 'inspection_score',
       'inspection_type', 'violation_id', 'violation_description',
       'risk_category'],
      dtype='object')

In [4]:
raw=raw.dropna(subset=['business_name', 'business_address'])

In [5]:
raw.shape

(50630, 14)

In [ ]:
# a 10-row mini test df
mni =raw.loc[50610:50619]

In [ ]:
mni.shape

In [ ]:
mni

In [ ]:
idc=clean_dirty_inspection(raw)

In [6]:
def token_set(name, value):
    output = fuzz.token_set_ratio(name, value)
    return output

def token_sort(name, value):
    output = fuzz.token_sort_ratio(name, value)
    return output

def clean_dirty_inspection(df):
    set_sort = np.vectorize(token_set)
    sort_sort = np.vectorize(token_sort)
    start = time.time()
    idct = {}
    iterator =0
    df = df[['business_name', 'business_address']]
    for zipcode, rows in df.groupby("business_postal_code"):
        name_list = list(rows['business_name'])
        addr_list = list(rows['business_address'])
        tuple_list = list(zip(name_list, addr_list))
        for t in tuple_list:
            name, address = t
            if name not in rows['business_name']:
                pass
            rows.loc[:, 'name_sim'] = set_sort(rows['business_name'], name)
            rows.loc[:, 'addr_sim'] = sort_sort(rows['business_address'], address)

            same = rows.loc[(rows['name_sim'] >= 80)]
            same = same.loc[(same['addr_sim'] >= 80)]
            ix = same.index.get_level_values(0)
            x=list(ix)
            if x != []:
                idct[iterator] = x
                iterator += 1
            rows = rows[~rows.isin(same).dropna()]
            df = df[~df.index.isin(same.index)]
    end = time.time()
    print(end-start)
    return idct

In [7]:
clean_dirty_inspection(raw[:1000])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


16.399317979812622


{0: [39, 153, 597, 821, 997],
 1: [57],
 2: [64, 65, 563, 667, 760],
 3: [80, 775],
 4: [88],
 5: [115],
 6: [146, 405, 742, 809, 814, 892],
 7: [147],
 8: [157],
 9: [159, 844, 891],
 10: [165],
 11: [174],
 12: [176],
 13: [187, 283, 336, 386, 444, 845],
 14: [226],
 15: [237],
 16: [257, 545],
 17: [289, 333, 720, 733, 868],
 18: [320, 328],
 19: [359],
 20: [375],
 21: [416, 417],
 22: [419],
 23: [451, 620],
 24: [482],
 25: [484],
 26: [534],
 27: [557],
 28: [674],
 29: [680, 851],
 30: [731],
 31: [743],
 32: [798],
 33: [897],
 34: [963],
 35: [3, 32, 673, 730],
 36: [7, 247, 869],
 37: [18, 529, 707],
 38: [19],
 39: [38, 695, 696],
 40: [41],
 41: [42, 70, 381, 400, 879, 904],
 42: [55],
 43: [62],
 44: [69, 240],
 45: [75, 759],
 46: [84],
 47: [93],
 48: [112, 870],
 49: [122, 443, 489, 637, 931],
 50: [137, 260, 949],
 51: [139, 805],
 52: [190, 310],
 53: [205],
 54: [222],
 55: [251, 797],
 56: [259, 264, 477, 716],
 57: [277],
 58: [282],
 59: [341],
 60: [346],
 61: [

In [8]:
clean_dirty_inspection(raw[:5000])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


177.89051699638367


{0: [4538, 4866],
 1: [4928],
 2: [39, 153, 597, 821, 997, 3491, 4760, 4794],
 3: [57, 1181, 1791, 2090],
 4: [64, 65, 563, 667, 760, 1823],
 5: [80, 775, 1132, 1690, 4045],
 6: [88],
 7: [115, 2272],
 8: [146, 405, 742, 809, 814, 892, 1070, 1077],
 9: [147],
 10: [157],
 11: [159, 844, 891, 1088, 3009, 3136, 3759, 3910, 4252, 4431],
 12: [165],
 13: [174],
 14: [176, 1194, 1202, 2026],
 15: [187, 283, 336, 386, 444, 845, 1007, 1114, 1377],
 16: [226],
 17: [237, 1695],
 18: [257, 545, 1096],
 19: [289, 333, 720, 733, 868, 1779, 1811, 2376, 3087, 3303, 3311, 3732, 3911],
 20: [320, 328, 3182, 3279, 3903, 4612, 4682],
 21: [359, 1338, 3596, 3758],
 22: [375, 2276, 2659],
 23: [416, 417],
 24: [419],
 25: [451, 620, 1988, 2808, 3201],
 26: [482],
 27: [484],
 28: [534, 1995, 2503, 3148, 4354],
 29: [557],
 30: [674],
 31: [680, 851],
 32: [731],
 33: [743, 3965, 4112],
 34: [798, 2593, 3859],
 35: [897, 1863, 2819, 4026, 4324, 4562],
 36: [963, 4227],
 37: [1010],
 38: [1031, 1343, 1764,

In [9]:
clean_dirty_inspection(raw[:10000])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


667.5896472930908


{0: [4538, 4866, 5751, 5986, 6266],
 1: [4928, 5003],
 2: [5597],
 3: [39, 153, 597, 821, 997, 3491, 4760, 4794],
 4: [57, 1181, 1791, 2090],
 5: [64, 65, 563, 667, 760, 1823, 9015],
 6: [80, 775, 1132, 1690, 4045, 5383, 5731],
 7: [88],
 8: [115, 2272, 7012, 7483],
 9: [146, 405, 742, 809, 814, 892, 1070, 1077],
 10: [147],
 11: [157],
 12: [159, 844, 891, 1088, 3009, 3136, 3759, 3910, 4252, 4431],
 13: [165, 5785, 9779, 9967],
 14: [174],
 15: [176, 1194, 1202, 2026],
 16: [187, 283, 336, 386, 444, 845, 1007, 1114, 1377, 8523, 8976, 9133, 9427],
 17: [226],
 18: [237, 1695],
 19: [257, 545, 1096],
 20: [289, 333, 720, 733, 868, 1779, 1811, 2376, 3087, 3303, 3311, 3732, 3911],
 21: [320, 328, 3182, 3279, 3903, 4612, 4682],
 22: [359, 1338, 3596, 3758, 8287, 8456, 8483, 8487, 8982],
 23: [375, 2276, 2659],
 24: [416, 417],
 25: [419],
 26: [451, 620, 1988, 2808, 3201, 5230],
 27: [482, 6239, 6291, 6438, 6468, 6593, 6595, 6739, 6743, 9157],
 28: [484],
 29: [534, 1995, 2503, 3148, 4354]

In [7]:
def link_record(df, idct):
    '''
    Using the dicitionary created, matching the unique resturant's business_id to index in both
    rest_df and insp_df
    '''
    start = time.time()
    insp_columns = ['inspection_id', 'inspection_date', 'inspection_score', 'inspection_type', 'violation_id', 'violation_description', 'risk_category']
    insp_df = df[insp_columns]
    rest_columns = ['business_name', 'business_address', 'business_city','business_state', 'business_postal_code', 'business_latitude', 'business_longitude', 'business_phone_number']                    
    rest_df = df[rest_columns]
    
    for k, v in idct.items():
        insp_df.loc[v, "business_id"]= int(k)
        rest_df.loc[v[-1], 'business_id']= int(k)
    rest_df = rest_df.dropna(subset=["business_id"])
    end = time.time()
    print(end-start)
    return insp_df, rest_df
    

In [ ]:
ins, res =link_record(raw, idc)

In [ ]:
ins

In [ ]:
res

In [93]:
def get_2str(df):
    '''
    Inputs:
        a pandas dataframe

    Returns: a list of tuples contains the three column values of each row 
            converted to a pandas Series
    '''
    str_lst =[]
    for i, row in df.iterrows():
        str_lst.append((row.business_name, row.business_address))
    tdf = pd.Series(str_lst)
    rv = pd.concat([df["business_postal_code"], tdf], axis=1)
    rv.columns = ['business_postal_code', 'tuple']
    return rv


In [35]:
def get_3str(df):
    '''
    Inputs:
        a pandas dataframe

    Returns: a list of tuples contains the three column values of each row 
            converted to a pandas Series
    '''
    str_lst =[]
    for i, row in df.iterrows():
        str_lst.append((row.business_name, row.business_address, row.business_postal_code))
    return pd.Series(str_lst)


In [149]:
t2 =get_2str(raw[:10])

In [150]:
t2

,business_postal_code,tuple
0,94109,"(Dar Bar Pakistani/Indian Cusine, 1412 Polk St)"
1,94110,"(DRAGON CITY BAKERY & CAFE, 2367 MISSION St)"
2,94110,"(Deli 23, 2449 23rd St)"
3,94103,"(L'acajou Bakery and Cafe, 498 09th St Ste. C)"
4,94108,"(Sweetheart Cafe, 909 Grant Ave)"
5,94108,"(Cafe New Honolulu, 888 Stockton St)"
6,94109,"(Aux Delices Vietnam, 2327 Polk St)"
7,94103,"(CAFE PICARO, 3120 16th St)"
8,94110,"(Jelly Donut, 3198 24th St)"
9,94110,"(Limon Peruvian Rotisserie, 1001 South Van Nes..."


In [105]:
t2 =t2.set_index('business_postal_code')

In [153]:
for _, group in t2.groupby('business_postal_code'):
    print(group.tuple)
    print("")

3    (L'acajou Bakery and Cafe, 498 09th St Ste. C)
7                       (CAFE PICARO, 3120 16th St)
Name: tuple, dtype: object

4        (Sweetheart Cafe, 909 Grant Ave)
5    (Cafe New Honolulu, 888 Stockton St)
Name: tuple, dtype: object

0    (Dar Bar Pakistani/Indian Cusine, 1412 Polk St)
6                (Aux Delices Vietnam, 2327 Polk St)
Name: tuple, dtype: object

1         (DRAGON CITY BAKERY & CAFE, 2367 MISSION St)
2                              (Deli 23, 2449 23rd St)
8                          (Jelly Donut, 3198 24th St)
9    (Limon Peruvian Rotisserie, 1001 South Van Nes...
Name: tuple, dtype: object



In [66]:
raw["business_postal_code"][:10]

0    94109
1    94110
2    94110
3    94103
4    94108
5    94108
6    94109
7    94103
8    94110
9    94110
Name: business_postal_code, dtype: object

In [67]:
r2 = pd.concat([raw["business_postal_code"][:10], s], axis=1)

In [68]:
r2

,business_postal_code,0
0,94109,"(Dar Bar Pakistani/Indian Cusine, 1412 Polk St)"
1,94110,"(DRAGON CITY BAKERY & CAFE, 2367 MISSION St)"
2,94110,"(Deli 23, 2449 23rd St)"
3,94103,"(L'acajou Bakery and Cafe, 498 09th St Ste. C)"
4,94108,"(Sweetheart Cafe, 909 Grant Ave)"
5,94108,"(Cafe New Honolulu, 888 Stockton St)"
6,94109,"(Aux Delices Vietnam, 2327 Polk St)"
7,94103,"(CAFE PICARO, 3120 16th St)"
8,94110,"(Jelly Donut, 3198 24th St)"
9,94110,"(Limon Peruvian Rotisserie, 1001 South Van Nes..."


In [77]:
raw.loc[:10,["business_postal_code", "business_name", "business_address"]]

,business_postal_code,business_name,business_address
0,94109,Dar Bar Pakistani/Indian Cusine,1412 Polk St
1,94110,DRAGON CITY BAKERY & CAFE,2367 MISSION St
2,94110,Deli 23,2449 23rd St
3,94103,L'acajou Bakery and Cafe,498 09th St Ste. C
4,94108,Sweetheart Cafe,909 Grant Ave
5,94108,Cafe New Honolulu,888 Stockton St
6,94109,Aux Delices Vietnam,2327 Polk St
7,94103,CAFE PICARO,3120 16th St
8,94110,Jelly Donut,3198 24th St
9,94110,Limon Peruvian Rotisserie,1001 South Van Ness Ave


In [31]:
for t in s:
    name, addr  =t
    print(name)
    print(addr)

Dar Bar Pakistani/Indian Cusine
1412 Polk St
DRAGON CITY BAKERY & CAFE
2367 MISSION St
Deli 23
2449 23rd St
L'acajou Bakery and Cafe
498 09th St Ste. C
Sweetheart Cafe
909 Grant Ave
Cafe New Honolulu
888 Stockton St
Aux Delices Vietnam
2327 Polk St
CAFE PICARO
3120 16th St
Jelly Donut
3198 24th St
Limon Peruvian Rotisserie
1001 South Van Ness Ave


In [ ]:
def get_tuple(a, b):
    '''
    Inputs:
        one row from each of the two parallel dataframe

    Returns: a single tuple contains three categories
    '''
    j = jellyfish.jaro_winkler(a, b)
    name_score = get_jw_category(j)

    return name_score

In [ ]:
THRESH1 = 0.8
THRESH2 = 1.0

def get_jw_category(j):
    '''
    Convert a Jaro-Winkler score into a categorical: low, medium. high
    using fixed thresholds.

    Inputs:
        j (double): value between 0 and 1 (inclusive)

    Returns: string
    '''
    if j < THRESH1:
        return "low"
    if j < THRESH2:
        return "medium"
    return "high"